**tf.tsv**

always predict A-coref as True and B-coref as False

In [94]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [95]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03



**closer.tsv**

Predict closer coreference to pronoun as True

In [22]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [23]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94



In [103]:
import pandas as pd

def print_sents(num_lines=10, datatype='train', random=True, datas={'manual':False, 'data':None}):
    
    if datas['manual'] == False:
        if datatype == 'train':
            data = "gap-development.tsv"
        elif datatype == 'validate':
            data = "gap-validation.tsv"
        elif datatype == 'test':
            data = "gap-test.tsv"
        else:
            data = "gap-development.tsv"

        lines = pd.read_csv('gap-development.tsv', sep='\t')
        if random:
            lines.sample(frac=1)
    else:
        lines = datas['data']

    count = 1
    for row in lines.iloc:

        sent = row['Text']
        pn, pn_id = row[['Pronoun', 'Pronoun-offset']]
        A, A_id = row[['A', 'A-offset']]
        B, B_id = row[['B', 'B-offset']]

        pn_len, A_len, B_len = len(pn), len(A), len(B)

        pn = "\033[94m" + pn + "\033[0m"
        A  = "\033[92m" + A  + "\033[0m" if row['A-coref'] else \
             "\033[91m" + A  + "\033[0m"
        B  = "\033[92m" + B  + "\033[0m" if row['B-coref'] else \
             "\033[91m" + B  + "\033[0m"

        indices = [(pn_id, pn, pn_len), (A_id, A, A_len), (B_id, B, B_len)]

        for index, content, length in sorted(indices, reverse=True):
            sent = sent[:index] + content + sent[index+length:]

        print(sent, end='\n\n')

        count += 1
        if count > num_lines:
            break
        

In [104]:
print_sents(random=True)

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.

He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.

He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated

In [71]:
for row in train_data.iloc:
    print(row)

ID
Text
Pronoun
Pronoun-offset
A
A-offset
A-coref
B
B-offset
B-coref
URL


In [19]:
import pandas as pd

In [96]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [61]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [97]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [98]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96



In [99]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

**relation.tsv**

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [11]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

In [12]:
tagged_sent

[('In', 'IN'),
 ('the', 'DT'),
 ('next', 'JJ'),
 ('pair', 'NN'),
 (',', ','),
 ('Mikhaylov', 'NNP'),
 ('equalled', 'VBD'),
 ('Grishin', 'NNP'),
 ("'s", 'POS'),
 ('time', 'NN'),
 ('and', 'CC'),
 ('they', 'PRP'),
 ('shared', 'VBD'),
 ('Olympic', 'NNP'),
 ('gold', 'NN'),
 ('.', '.')]

In [ ]:
from nltk.tree import Tree
sent_tree = Tree('S', 'PP', [Tree('NP') ])

In [95]:
### word that is inside paranthesis is False
train_data = pd.read_csv('gap-development.tsv', sep='\t')
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [101]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)
parenthesis_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
145,development-146,"Rocky intervenes, and after seemingly beating ...",her,430,Natasha,501,True,Supriya Pilgaonkar,526,False,http://en.wikipedia.org/wiki/Deewane_Huye_Paagal
175,development-176,When Herman and Fred are arrested and imprison...,his,286,Bruce Dern,243,False,Fred,263,True,http://en.wikipedia.org/wiki/Bloody_Mama
267,development-268,Drum grows up to be a fighter and is often for...,his,427,Hammond Maxwell,381,True,Oates,398,False,http://en.wikipedia.org/wiki/Drum_(1976_film)
332,development-333,Challenge of the Masters (Chinese: *******) is...,he,200,Gordon Liu,279,True,Chen Kuan Tai,321,False,http://en.wikipedia.org/wiki/Challenge_of_the_...
365,development-366,The family is financially supported by Cedric'...,she,315,Chrissie,190,False,Evangeline,227,True,http://en.wikipedia.org/wiki/Nanny_McPhee


In [114]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

Rocky intervenes, and after seemingly beating the living daylights out of the supposed thugs (the scene is similar to Jean-Claude Van Damme's introductory action sequence from Hard Target (1993)), introduces himself to Natasha, and proceeds to show himself off as a charming professional with a heart of gold. Somewhat awed, Natasha offers Rocky an invitation for tea later in the evening, which he accepts. When Rocky arrives at her villa later in the evening, he is confronted by several obstacles, Natasha's sister Sweety (Supriya Pilgaonkar), the jealous Tommy and the family's pet dog.

When Herman and Fred are arrested and imprisoned for petty theft charges, Ma takes over the group and leads Arthur and Lloyd on a bank robbery spree to gain enough money to get her sons out of jail. The gang is joined by a gunman named Kevin (Bruce Dern) who was Fred's cellmate during his incarceration (and his strongly implied lover).

Drum grows up to be a fighter and is often forced to bare-knuckle-box

In [115]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]

In [116]:
print_sents(datas={'manual':True, 'data':subject_data}, num_lines = 10)

Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar. She sang on the subsequent Deutsche Grammophon recording of the opera. For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice, and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007. She has since sung several parts and roles in John Adams' works, including the soprano part in El Ni*o, and the role of Kumudha in A Flowering Tree in the Peter Sellars production at the New Crowned Hope Festival in Vienna.

Sandra Collins is an American DJ. She got her start on the West Coast of the U.S. in Phoenix, Arizona and into residencies in Los Angeles, and eventually moved towards trance. She used American producers to give herself a unique sound. Collins performed for an estimated 80,000 people on the first night of Woodstock '99, and was the firs

In [138]:
subject_data.iloc[4]

ID                                                   development-16
Text              A colleague in the department run by Erwin Fri...
Pronoun                                                         she
Pronoun-offset                                                  295
A                                                      Alma Taggart
A-offset                                                        192
A-coref                                                       False
B                                                         McCulloch
B-offset                                                        219
B-coref                                                       False
URL                    http://en.wikipedia.org/wiki/Lucia_McCulloch
Name: 15, dtype: object

In [139]:
raw = subject_data.iloc[4]['Text']
words = nltk.word_tokenize(raw)
word_index = {}
index = 0
for i, word in enumerate(words):
    word_index[index] = i
    index += len(word)
    if not word in ['.', ',', "'s", '(', ')', "''", '``']:
        index += 1
    
    if word in ['*']:
        index -= 2

In [142]:
print(raw)
print(words)
print(word_index)

A colleague in the department run by Erwin Frink Smith, she also collaborated with botanist Nellie Adalesa Brown. McCulloch was born in Cincinnati, Ohio. She was the daughter of Robert S. and Alma Taggart (n*e Eveleth) McCulloch. In 1898, while studying biology at Florida Agricultural College, she was appointed head of the library, now part of the George A. Smathers Libraries.
['A', 'colleague', 'in', 'the', 'department', 'run', 'by', 'Erwin', 'Frink', 'Smith', ',', 'she', 'also', 'collaborated', 'with', 'botanist', 'Nellie', 'Adalesa', 'Brown', '.', 'McCulloch', 'was', 'born', 'in', 'Cincinnati', ',', 'Ohio', '.', 'She', 'was', 'the', 'daughter', 'of', 'Robert', 'S.', 'and', 'Alma', 'Taggart', '(', 'n', '*', 'e', 'Eveleth', ')', 'McCulloch', '.', 'In', '1898', ',', 'while', 'studying', 'biology', 'at', 'Florida', 'Agricultural', 'College', ',', 'she', 'was', 'appointed', 'head', 'of', 'the', 'library', ',', 'now', 'part', 'of', 'the', 'George', 'A.', 'Smathers', 'Libraries', '.']
{0: 

In [137]:
subjects_data = []
for row in subject_data.iloc:

    def id2word_index(words):
        word_index = {}
        index = 0
        for i, word in enumerate(words):
            word_index[index] = i
            index += len(word)
            if not word in ['.', ',', "'s", '(', ')', '"', '``']:
                index += 1
            if word in ['*']:
                index -= 2
        return word_index

    words = nltk.word_tokenize(row['Text'])
    tagged_words = nltk.pos_tag(words)
    word_index = id2word_index(words)
    
    def is_subject(text, word, word_id):
        word_length = len(word.split())
        word ,tag = tagged_words[word_index[word_id]+word_length]
        if tag.startswith('V'):
            return True
        else:
            return False

    print(row)
    if is_subject(row['Text'], row['A'], row['A-offset']) or \
       is_subject(row['Text'], row['B'], row['B-offset']):
            subjects_data.append(row)

subjects_data = pd.DataFrame(subjects_data, columns= subject_data.columns)

ID                                                    development-5
Text              Her Santa Fe Opera debut in 2005 was as Nuria ...
Pronoun                                                         She
Pronoun-offset                                                  437
A                                                 Kitty Oppenheimer
A-offset                                                        219
A-coref                                                       False
B                                                            Rivera
B-offset                                                        294
B-coref                                                        True
URL                     http://en.wikipedia.org/wiki/Jessica_Rivera
Name: 4, dtype: object
ID                                                    development-6
Text              Sandra Collins is an American DJ. She got her ...
Pronoun                                                         She
Pronoun-offset           

KeyError: 219